# Regression Analysis
---
- Author: Diego Inácio
- GitHub: [github.com/diegoinacio](https://github.com/diegoinacio)
- Notebook: [regression-analysis.ipynb](https://github.com/diegoinacio/data-science-notebooks/blob/master/Data-Analytics/regression-analysis.ipynb)
---
Analysis and implementation of some of the main *Regression* models.

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from regression_analysis__utils import *

## Linear Regression
---

In [ ]:
# Synthetic data 1
x, yA, yB, yC, yD = synthData1()

![linear regression correlation](output/regression_linear_correlation.png "Linear Regression Correlation")

### Simple Linear Regression
---
$$ \large
    y_i=mx_i+b
$$

Where **m** describes the angular coefficient (or line slope) and **b** the linear coefficient (or line y-intersept).

$$ \large
    m=\frac{\sum_i^n (x_i-\overline{x})(y_i-\overline{y})}{\sum_i^n (x_i-\overline{x})^2}
$$

$$ \large
    b=\overline{y}-m\overline{x}
$$

In [ ]:
class linearRegression_simple(object):
    def __init__(self):
        self._m = 0
        self._b = 0
    
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        X_ = X.mean()
        y_ = y.mean()
        num = ((X - X_)*(y - y_)).sum()
        den = ((X - X_)**2).sum()
        self._m = num/den
        self._b = y_ - self._m*X_
    
    def pred(self, x):
        x = np.array(x)
        return self._m*x + self._b

In [ ]:
lrs = linearRegression_simple()

In [ ]:
%%time

lrs.fit(x, yA)
yA_ = lrs.pred(x)

lrs.fit(x, yB)
yB_ = lrs.pred(x)

lrs.fit(x, yC)
yC_ = lrs.pred(x)

lrs.fit(x, yD)
yD_ = lrs.pred(x)

![linear regression prediction](output/regression_linear_pred.png "Linear Regression Prediction")

$$ \large
MSE=\frac{1}{n} \sum_i^n (Y_i- \hat{Y}_i)^2
$$

![linear regression residuals](output/regression_linear_residual.png "Linear Regression Residuals")

### Multiple Linear Regression
---
$$ \large
y=m_1x_1+m_2x_2+...+m_nx_n+b
$$

In [ ]:
class linearRegression_multiple(object):
    def __init__(self):
        self._m = 0
        self._b = 0
    
    def fit(self, X, y):
        X = np.array(X).T
        y = np.array(y).reshape(-1, 1)
        X_ = X.mean(axis = 0)
        y_ = y.mean(axis = 0)
        num = ((X - X_)*(y - y_)).sum(axis = 0)
        den = ((X - X_)**2).sum(axis = 0)
        self._m = num/den
        self._b = y_ - (self._m*X_).sum()
    
    def pred(self, x):
        x = np.array(x).T
        return (self._m*x).sum(axis = 1) + self._b

In [ ]:
lrm = linearRegression_multiple()

In [ ]:
%%time 
# Synthetic data 2
M = 10
s, t, x1, x2, y = synthData2(M)

# Prediction
lrm.fit([x1, x2], y)
y_ = lrm.pred([x1, x2])

![linear regression multiple](output/regression_linear_multiple_pred.png "Linear Regression Multiple")
![linear regression multiple residuals](output/regression_linear_multipla_residual.png "Linear Regression Multiple Residuals")

### Gradient Descent
---
$$ \large
    e_{m,b}=\frac{1}{n} \sum_i^n (y_i-(mx_i+b))^2
$$

To perform the gradient descent as a function of the error, it is necessary to calculate the gradient vector $\nabla$ of the function, described by:

$$ \large
\nabla e_{m,b}=\Big\langle\frac{\partial e}{\partial m},\frac{\partial e}{\partial b}\Big\rangle
$$

where:

$$ \large
\begin{aligned}
    \frac{\partial e}{\partial m}&=\frac{2}{n} \sum_{i}^{n}-x_i(y_i-(mx_i+b)), \\
    \frac{\partial e}{\partial b}&=\frac{2}{n} \sum_{i}^{n}-(y_i-(mx_i+b))
\end{aligned}
$$

In [ ]:
class linearRegression_GD(object):
    def __init__(self,
                 mo = 0,
                 bo = 0,
                 rate = 0.001):
        self._m = mo
        self._b = bo
        self.rate = rate
        
    def fit_step(self, X, y):
        X = np.array(X)
        y = np.array(y)
        n = X.size
        dm = (2/n)*np.sum(-x*(y - (self._m*x + self._b)))
        db = (2/n)*np.sum(-(y - (self._m*x + self._b)))
        self._m -= dm*self.rate
        self._b -= db*self.rate
        
    def pred(self, x):
        x = np.array(x)
        return self._m*x + self._b

In [ ]:
%%time
lrgd = linearRegression_GD(rate=0.01)

# Synthetic data 3
x, x_, y = synthData3()

iterations = 3072
for i in range(iterations):
    lrgd.fit_step(x, y)
y_ = lrgd.pred(x)

![gradient descent](output/regression_linear_gradDesc.gif "Gradient Descent")

## Logistic Regression
---
$$ \large
h_{\theta}(x)=g(\theta^Tx)=\frac{e^{\theta^Tx}}{1+e^{\theta^Tx}}=\frac{1}{1+e^{-\theta^Tx}}
$$

where:

$$ \large
\theta^Tx=
\begin{bmatrix}
    \theta_0 \\
    \theta_1 \\
    \vdots \\
    \theta_i
\end{bmatrix}
\begin{bmatrix}
    1 & x_{11} & \cdots & x_{1i} \\
    1 & x_{21} & \cdots & x_{2i} \\
    \vdots & \vdots & \ddots & \vdots \\
    1 & x_{n1} & \cdots & x_{ni}
\end{bmatrix}
$$

where:

- $\large h_\theta(x)$ is the hypothesis;
- $\large g(z)$ is the logistic function or <em>sigmoid</em>;
- $\large \theta_i$ is the parameters (or <em>weights</em>).

In [ ]:
def arraycast(f):
    '''
    Decorator for vectors and matrices cast
    '''
    def wrap(self, *X, y=[]):
        X = np.array(X)
        X = np.insert(X.T, 0, 1, 1)
        if list(y):
            y = np.array(y)[np.newaxis]
            return f(self, X, y)
        return f(self, X)
    return wrap

class logisticRegression(object):
    def __init__(self, rate=0.001, iters=1024):
        self._rate = rate
        self._iters = iters
        self._theta = None
    @property
    def theta(self):
        return self._theta
    def _sigmoid(self, Z):
        return 1/(1 + np.exp(-Z))
    def _dsigmoid(self, Z):
        return self._sigmoid(Z)*(1 - self._sigmoid(Z))
    @arraycast
    def fit(self, X, y=[]):
        self._theta = np.ones((1, X.shape[-1]))
        for i in range(self._iters):
            thetaTx = np.dot(X, self._theta.T)
            h = self._sigmoid(thetaTx)
            delta = h - y.T
            grad = np.dot(X.T, delta).T
            self._theta -= grad*self._rate
    @arraycast
    def pred(self, x):
        return self._sigmoid(np.dot(x, self._theta.T)) > 0.5

In [ ]:
# Synthetic data 5
x1, x2, y = synthData5()

![logistic regression data](output/regression_logistic_data.png "Logistic Regression Data")

In [ ]:
%%time
# Training
rlogb = logisticRegression(rate=0.001, iters=512)
rlogb.fit(x1, x2, y=y)
# rlogb.pred(x1, x2)

![logistic regression training](output/regression_logistic_gradDesc.gif "Logistic Regression Training")

To find the boundary line components:

$$ \large
    \theta_0+\theta_1 x_1+\theta_2 x_2=0
$$

Considering $\large x_2$ as the dependent variable:

$$ \large
    x_2=-\frac{\theta_0+\theta_1 x_1}{\theta_2}
$$

In [ ]:
# Prediction
w0, w1, w2 = rlogb.theta[0]
f = lambda x: - (w0 + w1*x)/w2

![regressão logística prediction](output/regression_logistic_pred.png "Logistic Regression Prediction")

## Polynomial Regression
---
Given the function:

$$ \large
    f(x)=x^3-3x^2+x+1+\epsilon
$$

In [ ]:
# Synthetic data 6
x, y = synthData6()

# Predicting with Linear Regression
lrs = linearRegression_simple()
lrs.fit(x, y)

![polynomial data and linear regression](output/regression_polynomial_linear.png "Polynomial data and Linear Regression")

### Algorithm
---
$$ \large
\vec{y}=\mathbf{X}\vec{\mathbf{\beta}}+\vec{\epsilon}
$$

where $\large \mathbf{X}$ (or $\large \mathbf{V}$) is the *Vandermonde's matrix* of the independent variable, parametrised by the maximum degree $\large m$, a response vector $\large \vec{y}$, a parameter vector $\large \vec{\mathbf{\beta}}$ and a random error vector $\large \vec{\epsilon}$. In the form of a system of linear equations, we have:

$$ \large
\begin{bmatrix}
    y_1 \\
    y_2 \\
    y_3 \\
    \vdots \\
    y_n
\end{bmatrix}
=
\begin{bmatrix}
    1 & x_1 & x_1^2 &\cdots & x_1^m \\
    1 & x_2 & x_2^2 & \cdots & x_2^m \\
    1 & x_3 & x_3^2 & \cdots & x_3^m \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    1 & x_n & x_n^2 & \cdots & x_n^m
\end{bmatrix}
\begin{bmatrix}
    \beta_1 \\
    \beta_2 \\
    \beta_3 \\
    \vdots \\
    \beta_m
\end{bmatrix}
+
\begin{bmatrix}
    \epsilon_1 \\
    \epsilon_2 \\
    \epsilon_3 \\
    \vdots \\
    \epsilon_n
\end{bmatrix}
$$

By means of the Least Squares Method, the estimated coefficient vector is given by:

$$ \large
\widehat{\vec{\mathbf{\beta}}}=(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\vec{y}
$$

In [ ]:
def arraycast(f):
    '''
    Decorador para conversão de vetores e matrizes
    '''
    def wrap(self, X, y=[]):
        X = np.array(X)
        if list(y):
            y = np.array(y)
            return f(self, X, y)
        return f(self, X)
    return wrap

class polynomialRegression(object):
    def __init__(self, degree=1):
        self._degree = degree
        self._beta = None
    @property
    def beta(self):
        return self._beta
    @arraycast
    def fit(self, X, y=[]):
        V = np.stack([X**i for i in range(self._degree + 1)], axis=0).T
        VTV = np.dot(V.T, V)
        VTV_i = np.linalg.inv(VTV)
        Vi = np.dot(VTV_i, V.T)
        self._beta = np.dot(Vi, y)
    @arraycast
    def pred(self, x):
        V = np.stack([x**i for i in range(self._degree + 1)], axis=0).T
        return np.dot(V, self._beta)

Notice that our class has an attribute called <em>degree</em> which is the maximum degree of our function $\large f(x)$. In our example it should be $\large m=3$.

In [ ]:
%%time
polreg = polynomialRegression(3)
polreg.fit(x, y=y)

![polynomial regression](output/regression_polynomial_pred.png "Polynomial Regression")